In [3]:
from flask import Flask, request, render_template_string
import json 

app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def submit_pick():
    if request.method == 'POST':
        username = request.form['username']
        tournament = request.form['tournament']
        pick = request.form['pick']
        
        # Insert the pick into the database
        insert_pick(username, tournament, pick)
        
        return f'Thank you! Your pick is {pick}.'
    
    usernames = [
        "Z", "Timmy Chips", "Goob", "C", "Steve", "Sus", "DV3", "Jlo", 
        "Ty Dolla $$$", "Ryan", "Mike D", "Mando", "Eddie Buckets", 
        "Garrett", "T Sully"
    ]
    
    tournaments = [
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic in The Palm Beaches", "Arnold Palmer Invitational presented by Mastercard", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
    ]
    
    return render_template_string('''
        <form method="post">
            Username: 
            <select name="username">
                {% for name in usernames %}
                    <option value="{{ name }}">{{ name }}</option>
                {% endfor %}
            </select><br>
            Tournament: 
            <select name="tournament">
                {% for tournament in tournaments %}
                    <option value="{{ tournament }}">{{ tournament }}</option>
                {% endfor %}
            </select><br>
            Golfer Pick: <input type="text" name="pick"><br>
            <input type="submit" value="Submit">
        </form>
    ''', usernames=usernames, tournaments=tournaments)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Jul/2024 22:23:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2024 22:23:36] "GET /favicon.ico HTTP/1.1" 404 -


In [17]:
from flask import Flask, request, render_template, redirect, url_for
import pandas as pd
import psycopg2

db_creds = pd.read_csv('/users/zack burnside/desktop/database_creds.csv')


app = Flask(__name__)

def insert_pick(username, tournament, pick):
    conn = psycopg2.connect(
            dbname=db_creds['dbname'][0],
            user=db_creds['user'][0],
            password=db_creds['password'][0],
            host=db_creds['host'][0],
            port="5433"
        )
    cursor = conn.cursor()
    cursor.execute("INSERT INTO PUBLIC.picks (username, tournament_name, player_pick) VALUES (%s, %s, %s)", 
                (username, tournament, pick))
    conn.commit()
    conn.close()

def fetch_leaderboard():
    conn = psycopg2.connect(
            dbname=db_creds['dbname'][0],
            user=db_creds['user'][0],
            password=db_creds['password'][0],
            host=db_creds['host'][0],
            port="5433"
    )
    cur = conn.cursor()
    cur.execute("SELECT DISTINCT user_name, SUM(prize_money::numeric) AS total_prize_money FROM public.weekly_tournament_results GROUP BY user_name ORDER BY total_prize_money DESC")
    leaderboard = cur.fetchall()
    cur.close()
    conn.close()
    return leaderboard

@app.route('/', methods=['GET', 'POST'])
def submit_pick():
    if request.method == 'POST':
        if 'submit_pick' in request.form:
            username = request.form['username']
            tournament = request.form['tournament']
            pick = request.form['pick']
            
            # Insert the pick into the database
            insert_pick(username, tournament, pick)
            
            return f'Thank you! Your pick is {pick}.'
        elif 'view_leaderboard' in request.form:
            return redirect(url_for('leaderboard'))
    
    usernames = [
        "Z", "Timmy Chips", "Goob", "C", "Steve", "Sus", "DV3", "Jlo", 
        "Ty Dolla $$$", "Ryan", "Mike D", "Mando", "Eddie Buckets", 
        "Garrett", "T Sully"
    ]
    
    tournaments = [
        "WM Phoenix Open", "The Genesis Invitational", "Mexico Open at Vidanta", 
        "Cognizant Classic in The Palm Beaches", "Arnold Palmer Invitational presented by Mastercard", 
        "THE PLAYERS Championship", "Valspar Championship", "Texas Children's Houston Open", 
        "Valero Texas Open", "Masters Tournament", "RBC Heritage", "THE CJ CUP Byron Nelson", 
        "Wells Fargo Championship", "PGA Championship", "Charles Schwab Challenge", 
        "RBC Canadian Open", "the Memorial Tournament presented by Workday", "U.S. Open", 
        "Travelers Championship", "Rocket Mortgage Classic", "John Deere Classic", 
        "Genesis Scottish Open", "The Open Championship", "3M Open", "Wyndham Championship", 
        "FedEx St. Jude Championship", "BMW Championship", "TOUR Championship"
    ]
    
    return render_template_string('''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
            }
            form {
                background: white;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            label {
                font-weight: bold;
                display: block;
                margin-top: 10px;
            }
            select, input[type="text"] {
                width: 100%;
                padding: 8px;
                margin-top: 5px;
                margin-bottom: 10px;
                border: 1px solid #ccc;
                border-radius: 4px;
            }
            input[type="submit"] {
                background-color: #4CAF50;
                color: white;
                padding: 10px 15px;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                margin-right: 10px;
            }
            input[type="submit"]:hover {
                background-color: #45a049;
            }
            .button-container {
                display: flex;
                justify-content: space-between;
            }
        </style>
        <form method="post">
            <label for="username">Username:</label>
            <select name="username" id="username">
                {% for name in usernames %}
                    <option value="{{ name }}">{{ name }}</option>
                {% endfor %}
            </select>
            <label for="tournament">Tournament:</label>
            <select name="tournament" id="tournament">
                {% for tournament in tournaments %}
                    <option value="{{ tournament }}">{{ tournament }}</option>
                {% endfor %}
            </select>
            <label for="pick">Golfer Pick:</label>
            <input type="text" name="pick" id="pick">
            <div class="button-container">
                <input type="submit" name="submit_pick" value="Submit Pick">
                <input type="submit" name="view_leaderboard" value="View Leaderboard">
            </div>
        </form>
    ''', usernames=usernames, tournaments=tournaments)

@app.route('/leaderboard')
def leaderboard():
    leaderboard_data = fetch_leaderboard()
    return render_template_string('''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }
            table {
                width: 50%;
                margin-top: 20px;
                border-collapse: collapse;
            }
            table, th, td {
                border: 1px solid #ccc;
            }
            th, td {
                padding: 10px;
                text-align: left;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
        </style>
        <h1>Leaderboard</h1>
        <table>
            <thead>
                <tr>
                    <th>Username</th>
                    <th>Total Prize Money</th>
                </tr>
            </thead>
            <tbody>
                {% for user, total in leaderboard_data %}
                    <tr>
                        <td>{{ user }}</td>
                        <td>${{ total }}</td>
                    </tr>
                {% endfor %}
            </tbody>
        </table>
        <a href="/" style="margin-top: 20px; display: inline-block; background-color: #4CAF50; color: white; padding: 10px 15px; text-decoration: none; border-radius: 4px;">Back to Form</a>
    ''', leaderboard_data=leaderboard_data)

if __name__ == '__main__':
    app.run(debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2024 15:49:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2024 15:49:48] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [17/Jul/2024 15:49:48] "GET /leaderboard HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2024 15:49:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2024 15:49:58] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [17/Jul/2024 15:49:58] "GET /leaderboard HTTP/1.1" 200 -
